In [1]:
%load_ext autoreload
%autoreload 2

from datasets.MSDWild import MSDWildChunks
from datasets.MSDWildOptimized import MSDWildChunksOptimized
import time
import os

In [2]:
S3_BUCKET_NAME = 'mmml-proj'
S3_VIDEO_DIR = 'preprocessed'
TRAIN_DATA_PATH = os.path.join("s3://", S3_BUCKET_NAME, S3_VIDEO_DIR)
TRAIN_RTTM_PATH = "few.train.rttm"
SUBSET = 0.025

In [ ]:
# Est Time: 27.75 minutes
start = time.time()
dataset = MSDWildChunks(data_path=S3_VIDEO_DIR,
                        data_bucket=S3_BUCKET_NAME,
                        partition_path=TRAIN_RTTM_PATH,
                        subset=SUBSET,
                        refresh_fileset=False)
end = time.time()
execution_time_minutes = (end - start) / 60

print(f"Execution time: {execution_time_minutes:.2f} minutes")

Directory has 523104 pairs


Loading Pair Metadata for Videos: 100%|██████████| 2476/2476 [02:44<00:00, 15.03it/s]


Loaded metadata for 1379378 pairs
Reducing number of pairs from 523104 to 13077


Loading Triplet Files:  30%|██▉       | 3888/13077 [08:39<31:54:43, 12.50s/it]

In [ ]:
start = time.time()
dataset = MSDWildChunksOptimized(data_path=S3_VIDEO_DIR,
                                 data_bucket=S3_BUCKET_NAME,
                                 partition_path=TRAIN_RTTM_PATH,
                                 subset=SUBSET,
                                 refresh_fileset=False)
end = time.time()
execution_time_minutes = (end - start) / 60

print(f"Execution time: {execution_time_minutes:.2f} minutes")